In [1]:
import pandas as pd

# Load the CSV file
file_path = '/path/to/your/file/youtube_data_from_kaggle.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataframe to understand its structure
data.head()


,#,NOMBRE,SEGUIDORES,TP,PAÍS,TEMA DE INFLUENCIA,ALCANCE POTENCIAL,GUARDAR,INVITAR A LA CAMPAÑA
0,1,NoCopyrightSounds @UC_aEa8K-EOJ3D6gOs7HcyNg,32.9M,-,Reino Unido,Música,9.9M,NaN,Ver Perfil
1,2,DanTDM @UCS5Oz6CHmeoF7vSad0qqXfw,26.3M,-,Reino Unido,NaN,7.9M,NaN,Ver Perfil
2,3,KSI @UCVtFOytbRpEvzLjvqGG5gxQ,24M,-,Reino Unido,NaN,7.2M,NaN,Ver Perfil
3,4,Mister Max @UC_8PAD0Qmi6_gpe77S1Atgg,23M,0.1%,Reino Unido,NaN,6.9M,NaN,Ver Perfil
4,5,Jelly @UC0DZmkupLYwc0yDsfocLh0A,22.9M,-,Reino Unido,NaN,6.9M,NaN,Ver Perfil


In [14]:
# Renaming the columns as requested
data.rename(columns={
    'NOMBRE': 'channel_name',
    'SEGUIDORES': 'total_subscribers',
    'ALCANCE POTENCIAL': 'total_views',
    'GUARDAR': 'total_videos'
}, inplace=True)

# Display the first few rows to confirm the changes
data.head()


,#,channel_name,total_subscribers,TP,PAÍS,TEMA DE INFLUENCIA,total_views,total_videos,INVITAR A LA CAMPAÑA
0,1,NoCopyrightSounds @UC_aEa8K-EOJ3D6gOs7HcyNg,32.9M,-,Reino Unido,Música,9.9M,NaN,Ver Perfil
1,2,DanTDM @UCS5Oz6CHmeoF7vSad0qqXfw,26.3M,-,Reino Unido,NaN,7.9M,NaN,Ver Perfil
2,3,KSI @UCVtFOytbRpEvzLjvqGG5gxQ,24M,-,Reino Unido,NaN,7.2M,NaN,Ver Perfil
3,4,Mister Max @UC_8PAD0Qmi6_gpe77S1Atgg,23M,0.1%,Reino Unido,NaN,6.9M,NaN,Ver Perfil
4,5,Jelly @UC0DZmkupLYwc0yDsfocLh0A,22.9M,-,Reino Unido,NaN,6.9M,NaN,Ver Perfil


Total videos are unavailable, so you need an 'API key' from the Google cloud console. Use ChatGPT to figure this out. This is not hard to do. 

In [ ]:
import requests
import pandas as pd

# Function to extract channel ID from the 'NOMBRE' column
def extract_channel_id(channel_name):
    return channel_name.split('@')[1]

# API key and base URL
api_key = 'you_should_keep_this_key_secret'
base_url = 'https://www.googleapis.com/youtube/v3/channels'

# Load the dataset
file_path = '/path/to/your/file/youtube_data_from_kaggle.csv'  # Replace with the path to your CSV file
data = pd.read_csv(file_path)

# Extract channel IDs
data['Channel_ID'] = data['NOMBRE'].apply(extract_channel_id)

# Function to get channel details for a given channel ID with error handling
def get_channel_details(channel_id, api_key):
    url = f'{base_url}?part=snippet,statistics&id={channel_id}&key={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        json_response = response.json()
        if 'items' in json_response and len(json_response['items']) > 0:
            item = json_response['items'][0]
            snippet = item['snippet']
            statistics = item['statistics']
            return {
                'Channel_Name': snippet['title'],
                'Subscribers': statistics.get('subscriberCount', None),
                'Total_Views': statistics.get('viewCount', None),
                'Total_Videos': statistics.get('videoCount', None)
            }
        else:
            print(f"No items found for channel ID: {channel_id}")
            return None
    else:
        print(f"Error fetching data for channel ID: {channel_id}, Status Code: {response.status_code}")
        return None

# Fetch channel details for each channel and add them to the dataframe
details = data['Channel_ID'].apply(lambda x: get_channel_details(x, api_key))

# Create new columns for the details
data['Channel_Name'] = details.apply(lambda x: x['Channel_Name'] if x else None)
data['Subscribers'] = details.apply(lambda x: x['Subscribers'] if x else None)
data['Total_Views'] = details.apply(lambda x: x['Total_Views'] if x else None)
data['Total_Videos'] = details.apply(lambda x: x['Total_Videos'] if x else None)

# Save the updated dataset to a new CSV file
output_file_path = '/path/to/your/file/youtube_data_with_total_videos.csv'  # Replace with your desired output path
data.to_csv(output_file_path, index=False)

output_file_path
